In [1]:
'''Celery task for scrapping data'''
import os
import re
from celery.task.schedules import crontab
from celery.decorators import periodic_task
from celery.utils.log import get_task_logger
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
# from django.http import HttpResponse
from jkm import celery_app
import requests
from bs4 import BeautifulSoup

In [99]:
page = requests.get("https://www.jumia.co.ke/")
soup = BeautifulSoup(page.content, 'html.parser')
nav = soup.select('.itm')

categories = []
all_products = []
for item in nav:
    if item.get('href'):
        if item['href'].startswith("/") and item['href'].endswith('/') and len(item.select('span.text')):
            categories.append({
                'name': item.select('span.text')[::-1][-1].text,
                'link': item['href'],
            })
total_length = 0
for category in categories:
    link = f"https://www.jumia.co.ke{category['link']}?page=1"
    soup = scrap_data_jumia_categories(category, link)
    pages = get_pagination_data(soup)
    for i in range(1, int(pages)+1):
        link = f"https://www.jumia.co.ke{category['link']}?page={i}"
        
        page = requests.get(link)
        soup_ = BeautifulSoup(page.content, 'html.parser')
        if soup_:
            products = soup.find_all(class_="sku")
            
            for product in products:
                # print(product)
                try:
                    old_price = find_prices(product)[0]
                    new_price = find_prices(product)[1]
                    discount_percentage = find_prices(product)[2]
                    total_ratings = find_ratings(product)[0]
                    avg_rating = find_ratings(product)[1]
                    brand = find_name(product)[0]
                    product_name = find_name(product)[1]
                    link = find_links(product)
                    image = find_image(product)
                    discount = float(old_price)-float(new_price)
                except:
                    pass

                print(old_price, new_price)


1570 999
1570 999
410 328
450 282
95 0
83 0
15999 7999
15999 7999
4200 2685
1050 893
400 0
300 240
160 0
160 0
650 520
260 234
2900 0
950 807
3499 3498
3499 3498
500 489
300 275
700 500
1950 1550
450 382
450 382
2500 1799
300 0
1050 893
2000 1500
450 382
450 382
1000 0
5 0
3300 2799
395 277
1250 1000
1250 1000
750 392
300 210
280 238
300 180
2000 1500
2000 1500
2680 1846
1850 999
120 103
500 475
1570 999
1570 999
410 328
450 282
95 0
83 0
15999 7999
15999 7999
4200 2685
1050 893
400 0
300 240
160 0
160 0
650 520
260 234
2900 0
950 807
3499 3498
3499 3498
500 489
300 275
700 500
1950 1550
450 382
450 382
2500 1799
300 0
1050 893
2000 1500
450 382
450 382
1000 0
5 0
3300 2799
395 277
1250 1000
1250 1000
750 392
300 210
280 238
300 180
2000 1500
2000 1500
2680 1846
1850 999
120 103
500 475
1570 999
1570 999
410 328
450 282
95 0
83 0
15999 7999
15999 7999
4200 2685
1050 893
400 0
300 240
160 0
160 0
650 520
260 234
2900 0
950 807
3499 3498
3499 3498
500 489
300 275
700 500
1950 1550
450 38

760 0
971 874
150 0
1740 1396
1740 1396
483 363
500 371
241 189
160 125
895 761
895 761
227 114
249 0
118 0
187 149
126 0
126 0
474 387
90 45
270 0
124 109
1100 924
1100 924
29 0
19 0
83 0
400 299
600 440
600 440
230 205
1475 1106
3600 1250
800 750
155 140
155 140
131 0
131 0
484 0
506 480
1939 1357
1939 1357
2000 1250
99 49
119 113
125 106
180 144
180 144
200 150
760 0
971 874
150 0
1740 1396
1740 1396
483 363
500 371
241 189
160 125
895 761
895 761
227 114
249 0
118 0
187 149
126 0
126 0
474 387
90 45
270 0
124 109
1100 924
1100 924
29 0
19 0
83 0
400 299
600 440
600 440
230 205
1475 1106
3600 1250
800 750
155 140
155 140
131 0
131 0
484 0
506 480
1939 1357
1939 1357
2000 1250
99 49
119 113
125 106
180 144
180 144
200 150
760 0
971 874
150 0
1740 1396
1740 1396
483 363
500 371
241 189
160 125
895 761
895 761
227 114
249 0
118 0
187 149
126 0
126 0
474 387
90 45
270 0
124 109
1100 924
1100 924
29 0
19 0
83 0
400 299
600 440
600 440
230 205
1475 1106
3600 1250
800 750
155 140
155 140
1

1400 0
1400 0
1740 1400
65 0
65 0
1400 1290
1049 1030
1049 1030
1400 1290
2400 1096
999 0
1200 619
499 248
499 248
1200 772
2000 1030
1090 0
1300 819
1500 998
1500 998
225 0
1250 0
1149 950
304 0
294 0
294 0
7929 0
1479 1044
19500 9500
1199 914
2400 1096
2400 1096
3800 1998
1000 419
1025 0
3000 1998
1400 0
1400 0
309 0
6000 4400
705 0
1400 0
1599 1399
1599 1399
1740 0
1150 0
1800 1080
1400 0
1400 0
1400 0
1740 1400
65 0
65 0
1400 1290
1049 1030
1049 1030
1400 1290
2400 1096
999 0
1200 619
499 248
499 248
1200 772
2000 1030
1090 0
1300 819
1500 998
1500 998
225 0
1250 0
1149 950
304 0
294 0
294 0
7929 0
1479 1044
19500 9500
1199 914
2400 1096
2400 1096
3800 1998
1000 419
1025 0
3000 1998
1400 0
1400 0
309 0
6000 4400
705 0
1400 0
1599 1399
1599 1399
1740 0
1150 0
1800 1080
1400 0
1400 0
1400 0
1740 1400
65 0
65 0
1400 1290
1049 1030
1049 1030
1400 1290
2400 1096
999 0
1200 619
499 248
499 248
1200 772
2000 1030
1090 0
1300 819
1500 998
1500 998
225 0
1250 0
1149 950
304 0
294 0
294 0
79

30000 16030
12999 8599
5999 5499
10000 6340
10000 6340
6999 6799
20999 17999
15999 11499
23999 20299
15000 9999
15000 9999
15499 12999
26599 20999
15999 12489
23999 19299
5999 5499
5999 5499
7999 7499
5599 5099
17599 15999
15999 11499
35000 27999
35000 27999
4999 2899
7999 7499
10590 6299
12000 10599
5999 5499
5999 5499
11999 9849
15999 15799
11299 9999
4999 2899
10899 10599
10899 10599
6399 5399
17599 15999
10599 9999
18000 9416
10599 7699
10599 7699
36000 30699
30000 14999
12599 11599
3500 2299
5099 4799
5099 4799
10599 9999
30000 16030
12999 8599
5999 5499
10000 6340
10000 6340
6999 6799
20999 17999
15999 11499
23999 20299
15000 9999
15000 9999
15499 12999
26599 20999
15999 12489
23999 19299
5999 5499
5999 5499
7999 7499
5599 5099
17599 15999
15999 11499
35000 27999
35000 27999
4999 2899
7999 7499
10590 6299
12000 10599
5999 5499
5999 5499
11999 9849
15999 15799
11299 9999
4999 2899
10899 10599
10899 10599
6399 5399
17599 15999
10599 9999
18000 9416
10599 7699
10599 7699
36000 30699

3500 1269
1690 955
2599 655
3000 654
18700 13895
18700 13895
9999 7795
10499 9795
7999 5900
3599 999
2600 1439
2600 1439
7399 0
25990 16495
11799 0
11500 6139
1050 0
1050 0
12999 10475
2825 2599
4299 3869
3800 1899
24999 22999
24999 22999
35999 29499
35000 0
39500 29999
49000 26799
32000 22539
32000 22539
17999 13999
35999 31000
18999 15999
33999 30999
1399 1199
1399 1199
1799 1199
700 695
49000 48585
27999 25499
27999 26500
27999 26500
35000 0
1250 625
1160 675
1299 999
1500 549
1500 549
849 399
399 379
3999 3299
699 649
399 369
399 369
699 499
1500 569
1800 976
1399 569
620 0
620 0
799 499
499 479
29999 22499
1500 1347
37499 35999
37499 35999
15999 12499
1499 550
12999 0
24000 19999
24999 22999
24999 22999
35999 29499
35000 0
39500 29999
49000 26799
32000 22539
32000 22539
17999 13999
35999 31000
18999 15999
33999 30999
1399 1199
1399 1199
1799 1199
700 695
49000 48585
27999 25499
27999 26500
27999 26500
35000 0
1250 625
1160 675
1299 999
1500 549
1500 549
849 399
399 379
3999 3299
6

KeyboardInterrupt: 

In [71]:
def persist_to_db(soup, category, total_length, all_products):
    '''
    Save items to the database
    '''
    products = soup.find_all(class_="sku")
    product_details = sort_product_details_out(products, category)
    total_length += product_details['length']
    all_products.append(product_details['prods'])

In [92]:
def find_prices(product):
    '''
    Find prices and discounts from product
    '''
    for a in product.find_all(class_='price-container'):
        x = a.find_all(text=True)
        old_price = 0
        new_price = 0
        discount_percentage = 0
        if x[0] != ' ':
            old_price = x[-3].replace(',', '')
            new_price = x[-8].replace(',', '')
            discount_percentage = float(x[0].strip('%')) * -1
        else:
            old_price = x[-5].replace(',', '')
        return [old_price, new_price, discount_percentage]
